In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import re
import warnings
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
import cv2

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Ellipse, Patch
#plt.rcParams["font.family"] = "DejaVu Serif"
plt.rcParams.update({'text.usetex': True, "font.family": "sans-serif", 'font.size': 18})
plt.style.use("tableau-colorblind10")  # [#006BA4, #FF800E, #ABABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
plt.rcParams['axes.axisbelow'] = True

import numpy as np
from numba import njit, prange
import pandas as pd
from PIL import Image
import rasterio
import xarray as xr

In [ ]:
warnings.filterwarnings(action='ignore', category=rasterio.errors.NotGeoreferencedWarning, module='rasterio')
warnings.filterwarnings(action='ignore', category=RuntimeWarning)

In [ ]:
# Load pandora imports
import pandora
pandora.setup_logging(True)

In [ ]:
# Load plugins
pandora.import_plugin()

### Defining functions for loading Middlebury GT and for correctly plotting greyscale images.

In [ ]:
def process_pfm(disp_path):
    """Read Middlebury disparity map and calculate depth map.
    http://davis.lbl.gov/Manuals/NETPBM/doc/pfm.html
    """
    disp = cv2.imread(disp_path, cv2.IMREAD_UNCHANGED)
    with open(disp_path, "rb") as pfm_file:
        header = pfm_file.readline().decode().rstrip()
        channels = 3 if header == "PF" else 1

        dim_match = re.match(r"^(\d+)\s(\d+)\s$", pfm_file.readline().decode("utf-8"))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception("Malformed PFM header.")

        scale = float(
            pfm_file.readline().decode().rstrip()
        )  # read disparity scale factor
        if scale < 0:
            endian = "<"  # littel endian
            scale = -scale
        else:
            endian = ">"  # big endian

    disp = disp * scale
    return disp

def normalize_for_imshow(img: np.ndarray):
    img_normalized = np.zeros(img.shape, dtype=img.dtype)
    for band in range(img.shape[-1]):
        min_band, max_band = np.nanquantile(img[:,:,band], [0.01, 0.99])
        img_normalized[:,:,band] = np.clip(img[:,:,band], min_band, max_band)
        img_normalized[:,:,band] = (img_normalized[:,:,band] - np.nanmin(img_normalized[:,:,band])) / (np.nanmax(img_normalized[:,:,band]) - np.nanmin(img_normalized[:,:,band]))
    return img_normalized

def crop_img(path):
    img = np.array(Image.open(path))
    img_sum = img.sum(axis=2)
    for k in range(img.shape[0]):
        if (img_sum[k, :] != (255*4)).any():
            top = k
            break
    for k in range(img.shape[0]):
        if (img_sum[img.shape[0] - 1 - k, :] != (255*4)).any():
            bottom = img.shape[0] - 1 - k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, k] != (255*4)).any():
            left = k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, img.shape[1] - 1 - k] != (255*4)).any():
            right = img.shape[1] - 1 - k
            break
    img = img[top:bottom+1, left:right+1]
    Image.fromarray(img).save(path)

In [ ]:
from numba import njit, prange

def compute_overstimation(true_disp, disp_map, lefts, rights, int_inf, int_sup):
    n = lefts.shape[0]
    output = np.zeros(n, dtype=np.float64)
    for k in prange(n):
        if (np.nanmax(true_disp[lefts[k, 0], lefts[k, 1]:rights[k,1]+1]) > np.nanmax(int_sup[lefts[k, 0], lefts[k, 1]:rights[k,1]+1])) |\
           (np.nanmin(true_disp[lefts[k, 0], lefts[k, 1]:rights[k,1]+1]) < np.nanmax(int_inf[lefts[k, 0], lefts[k, 1]:rights[k,1]+1])):
            output[k] = np.nan
        else:
            output[k] = np.nanmax(np.abs(true_disp[lefts[k, 0], lefts[k, 1]:rights[k,1]+1] - disp_map[lefts[k, 0], lefts[k, 1]:rights[k,1]+1])) \
            / (int_sup[lefts[k, 0], lefts[k, 1]]-int_inf[lefts[k, 0], lefts[k, 1]])
    return output

In [ ]:
out_path = "/work/CAMPUS/users/malinoro/outputs/These/IMG/"

blue="#006BA4"
orange="#FF800E"
gray="#ABABAB"
dark_gray="#595959"
gray_blue="#5F9ED1"
brown="#C85200"
third_gray="#898989"
light_blue="#A2C8EC"
light_orange="#FFBC79"
light_gray="#CFCFCF"

In [ ]:
def load_data_mtp(output_path):
    cfg_file_path = os.path.join(output_path, "cfg/config.json")
    with open(cfg_file_path, "r") as f:
        cfg_lines = f.readlines()
    for l in cfg_lines:
        if '"disp_min"' in l:
            d_min = float(l.split(': ')[1].split(",")[0].split("\n")[0])
        elif '"disp_max"' in l:
            d_max = float(l.split(': ')[1].split(",")[0].split("\n")[0])
    
    true_disp = rasterio.open(os.path.join(output_path, "left_epipolar_disp.tif")).read(1)
    disp_map = rasterio.open(os.path.join(output_path, "left_disparity.tif")).read(1)
    
    disp_conf = rasterio.open(os.path.join(output_path, "left_confidence_measure.tif"))
    int_90_inf = disp_conf.read(4)
    int_90_sup = disp_conf.read(5)
    disp_map[disp_map<int_90_inf] = int_90_inf[disp_map<int_90_inf]
    disp_map[disp_map>int_90_sup] = int_90_sup[disp_map>int_90_sup]
    
    validity_map = rasterio.open(os.path.join(output_path, "left_validity_mask.tif")).read(1)
    validity_bitmap = (validity_map & (1<<11))>0
    invalid_mask = (validity_map & 0b01111000011)>0
    
    valid_data = ~((true_disp==0) | (np.isnan(disp_map)) | (np.isnan(int_90_inf)) | (np.isnan(int_90_sup)) | invalid_mask)
    #true_disp[~valid_data] = np.nan
    #disp_map[~valid_data] = np.nan
    #int_90_inf[~valid_data] = np.nan
    #int_90_sup[~valid_data] = np.nan
    return d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data

def load_data_middlebury(output_path):
    cfg_file_path = os.path.join(output_path, "cfg/config.json")
    with open(cfg_file_path, "r") as f:
        cfg_lines = f.readlines()
    for l in cfg_lines:
        if '"disp_min"' in l:
            d_min = float(l.split(': ')[1].split(",")[0].split("\n")[0])
        elif '"disp_max"' in l:
            d_max = float(l.split(': ')[1].split(",")[0].split("\n")[0])
    
    gt_path = os.path.dirname(str([l.split('"') for l in cfg_lines if '"img_left": ' in l][0][-2]))
    year = gt_path.split("/")[-2]

    if year in ["2003", "2005", "2006"]:
        scale_gt = 4 if year=="2003" else 3
        true_disp = - np.asarray(Image.open(os.path.join(gt_path, "disp2.png"))).astype(float) / scale_gt
    elif year in ["2014", "2021"]:
        true_disp = -process_pfm(os.path.join(gt_path, "disp0.pfm"))
        true_disp[true_disp == -np.inf] = 0
    disp_map = rasterio.open(os.path.join(output_path, "left_disparity.tif")).read(1)
    
    disp_conf = rasterio.open(os.path.join(output_path, "left_confidence_measure.tif"))
    int_90_inf = disp_conf.read(4)
    int_90_sup = disp_conf.read(5)
    disp_map[disp_map<int_90_inf] = int_90_inf[disp_map<int_90_inf]
    disp_map[disp_map>int_90_sup] = int_90_sup[disp_map>int_90_sup]
    
    validity_map = rasterio.open(os.path.join(output_path, "left_validity_mask.tif")).read(1)
    validity_bitmap = (validity_map & (1<<11))>0
    invalid_mask = (validity_map & 0b01111000011)>0
    
    valid_data = ~((true_disp==0) | (np.isnan(disp_map)) | (np.isnan(int_90_inf)) | (np.isnan(int_90_sup)) | invalid_mask)
    #true_disp[~valid_data] = np.nan
    #disp_map[~valid_data] = np.nan
    #int_90_inf[~valid_data] = np.nan
    #int_90_sup[~valid_data] = np.nan
    
    return d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data

# Investigation

### Defining the scene, Cost function used, input and output paths.
* For Middlebury data, `year` should be in [2003, 2005, 2006, 2014, 2021]
* For Jacksonville or Montepellier data, `year` is "".

`scene` is the name of the scene. For Middlebury it is "cones" for example. For Montpellier or Jacksonville it is something like "MTP_144".
Check `/work/CAMPUS/users/malinoro/Data/Satellite` and `/work/CAMPUS/users/malinoro/Data/MiddleBury` to see which scenes are available

In [ ]:
#root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/"
#load_data = load_data_middlebury
#list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/CENSUS") if os.path.isdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/CENSUS/{k}")]


list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/CENSUS") if "MTP_"==k[:4]]
root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/"
load_data = load_data_mtp

In [ ]:
iterables = [["CENSUS", "MCCNN"], ["acc", "s_rel", "o_rel", "eps", "amb_area", "d_1"]]
columns = pd.MultiIndex.from_product(iterables, names=["cost_function", "metric"])
df = pd.DataFrame(columns=columns, index=list_scenes)

In [ ]:
for cost_function in ['CENSUS', 'MCCNN']:
    print()
    print(cost_function)
    for i, scene in enumerate(list_scenes):
        print(f"\r{i+1}/{len(list_scenes)}    ", end="")
        output_path = os.path.join(root_gt, cost_function, scene)
        d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data = load_data(output_path)
        
        amb_area = validity_bitmap[valid_data].sum() / valid_data.sum()
        correct_int = (true_disp<=int_90_sup) & (true_disp>=int_90_inf)

        d_1 = (np.abs(true_disp - disp_map)[valid_data] <1).sum() / valid_data.sum()
        
        acc = correct_int[valid_data].sum() / valid_data.sum()
        s_rel = np.nanmedian((int_90_sup- int_90_inf)[valid_data&(~validity_bitmap)]/(d_max-d_min))
        eps = np.nanmedian(np.minimum(np.abs(true_disp-int_90_inf)[((~correct_int)&valid_data)], np.abs(true_disp-int_90_sup)[((~correct_int)&valid_data)])) / (d_max-d_min)

        diffs = np.diff(np.hstack([np.zeros((validity_bitmap.shape[0], 1), dtype=np.bool_), validity_bitmap, np.zeros((validity_bitmap.shape[0], 1), dtype=np.bool_)]).astype(int))
        lefts, rights = np.argwhere(diffs==1), np.argwhere(diffs==-1) - np.array([0, 1])
        true_disp[~valid_data] = np.nan
        disp_map[~valid_data] = np.nan
        int_90_inf[~valid_data] = np.nan
        int_90_sup[~valid_data] = np.nan
        o_rel = 1-compute_overstimation(true_disp, disp_map, lefts, rights, int_90_inf, int_90_sup)

        mask_orel = ~np.isnan(o_rel)
        
        arg_sort = np.argsort(o_rel[mask_orel])
        o_rel = o_rel[mask_orel][arg_sort]

        counts = rights[mask_orel, 1] - lefts[mask_orel, 1] + 1
        counts = np.cumsum(counts[arg_sort])/np.sum(counts)


        median_index = np.max(np.argwhere(counts<=0.5))
        if median_index == 0.5:
            o_rel_med = o_rel[median_index]
        else:  # Simple interpolation of the median value.
            o_rel_med = (o_rel[median_index+1] - o_rel[median_index]) / (counts[median_index+1] - counts[median_index]) * (0.5 - counts[median_index]) + o_rel[median_index]

        
        df.loc[scene, (cost_function, ["acc", "s_rel", "o_rel", "eps", "amb_area", "d_1"])] = [acc, s_rel, o_rel_med, eps, amb_area, d_1]


In [ ]:
#df.to_csv("/work/CAMPUS/users/malinoro/outputs/These/MTP_metrics.csv")

# Statistics

In [ ]:
df_middlebury = pd.read_csv("/work/CAMPUS/users/malinoro/outputs/These/Middlebury_metrics.csv", header=[0,1], index_col=[0])
df_mtp = pd.read_csv("/work/CAMPUS/users/malinoro/outputs/These/MTP_metrics.csv", header=[0,1], index_col=[0])

In [ ]:
mtp_scenes =['MTP_1', 'MTP_100', 'MTP_120', 'MTP_127', 'MTP_132', 'MTP_151',
       'MTP_152', 'MTP_153', 'MTP_154', 'MTP_16', 'MTP_173', 'MTP_174',
       'MTP_176', 'MTP_198', 'MTP_2', 'MTP_201', 'MTP_21', 'MTP_223',
       'MTP_236', 'MTP_237', 'MTP_242', 'MTP_245', 'MTP_246', 'MTP_249',
       'MTP_252', 'MTP_254', 'MTP_259', 'MTP_261', 'MTP_262', 'MTP_263',
       'MTP_264', 'MTP_265', 'MTP_267', 'MTP_270', 'MTP_271', 'MTP_272',
       'MTP_274', 'MTP_275', 'MTP_276', 'MTP_284', 'MTP_285', 'MTP_286',
       'MTP_287', 'MTP_291', 'MTP_292', 'MTP_293', 'MTP_294', 'MTP_295',
       'MTP_296', 'MTP_297', 'MTP_298', 'MTP_3', 'MTP_301', 'MTP_302',
       'MTP_303', 'MTP_304', 'MTP_305', 'MTP_306', 'MTP_307', 'MTP_308',
       'MTP_313', 'MTP_314', 'MTP_315', 'MTP_316', 'MTP_317', 'MTP_318',
       'MTP_319', 'MTP_322', 'MTP_323', 'MTP_324', 'MTP_325', 'MTP_327',
       'MTP_328', 'MTP_329', 'MTP_43', 'MTP_44', 'MTP_6', 'MTP_88',
       'MTP_90', 'MTP_95']  #Those scene do not have too much differences between gt and left iamges

In [ ]:
df_mtp = df_mtp.iloc[[k in mtp_scenes for k in df_mtp.index]]
df_mtp.sort_values(by=("CENSUS", "acc"), ascending=False, inplace=True)
df_mtp

In [ ]:
year_dict = {}
years = ["2003", "2005", "2006", "2014", "2021"]
for year in years:
    scenes = os.listdir(f"/work/CAMPUS/users/malinoro/Data/MiddleBury/{year}")
    for scene in scenes:
        year_dict[scene] = year

In [ ]:
year= "2005"
mask_year = [year_dict[k] == year for k in df_middlebury.index]
df_middlebury[mask_year]

In [ ]:
def print_table(metric_key, cost_function):
    line = ""
    for year in ["2003", "2005", "2006", "2014", "2021"]:
        mask_year = [year_dict[k] == year for k in df_middlebury.index]

        line += r"$"
        line += str(np.round(np.nanmean(df_middlebury[mask_year][(cost_function, metric_key)])*100, 1))
        line += r"\%$ & "

    line = line[:-3]
    line += r"\\"
    return line

In [ ]:
print(r"""\begin{table}[h!]
\centering
\renewcommand{\arraystretch}{1.5}
\begin{tabular}{|c|c||c|c|c|c|c|}
\cline{2-7}
\rowcolor{lightgray}
\multicolumn{1}{c|}{\cellcolor{white}}& Year & 2003 & 2005 & 2006 & 2014 & 2021 \\ \hline

\rowcolor{color_census}
\cellcolor{white} & CENSUS & """ +\
print_table("acc", "CENSUS") +\
r"""\cline{2-7}

\rowcolor{color_mccnn}
\multirow{-2}{*}{\cellcolor{white} $acc$ $\uparrow$} & MC-CNN & """ +\
print_table("acc", "MCCNN") +\
r"""

\rowcolor{color_census}\hline
\cellcolor{white} & CENSUS & """ +\
print_table("eps", "CENSUS") +\
r"""\cline{3-7} 

\rowcolor{color_mccnn}
\multirow{-2}{*}{\cellcolor{white} $\epsilon_{~~~}$ $\downarrow$} & MC-CNN & """ +\
print_table("eps", "MCCNN") +\
r"""

\rowcolor{color_census}\hline
\cellcolor{white} & CENSUS & """ +\
print_table("s_rel", "CENSUS") +\
r"""\cline{3-7} 

\rowcolor{color_mccnn}
\multirow{-2}{*}{\cellcolor{white} $s_{rel}$ $\downarrow$} & MC-CNN & """ +\
print_table("s_rel", "MCCNN") +\
r"""

\rowcolor{color_census}\hline
\cellcolor{white} & CENSUS & """ +\
print_table("o_rel", "CENSUS") +\
r"""\cline{3-7} 

\rowcolor{color_mccnn}
\multirow{-2}{*}{\cellcolor{white} $o_{rel}$ $\downarrow$} & MC-CNN & """ +\
print_table("o_rel", "MCCNN") +\
r"""

\rowcolor{color_census}\hline
\cellcolor{white} & CENSUS & """ +\
print_table("amb_area", "CENSUS") +\
r"""\cline{3-7} 

\rowcolor{color_mccnn}
\multirow{-2}{*}{\cellcolor{white} $p_{amb}$ $\downarrow$} & MC-CNN & """ +\
print_table("amb_area", "MCCNN") +\
r"""\hline


\end{tabular}
\renewcommand{\arraystretch}{1}
\caption{Average metrics over the different Middlebury datasets, depending on the cost function. Up arrows indicate that the optimal score is $100\%$, and $0\%$ for down arrows.}\label{tab:metric_average}
\end{table}""")

## Accuracy

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_middlebury[("CENSUS", "acc")], bins=50, range=(0.5,1))
mccnn_bin, mccnn_edges = np.histogram(df_middlebury[("MCCNN", "acc")], bins=50, range=(0.5,1))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.axvline(0.9, color="k", linestyle=":")
ax.set_ylabel("\(\mathrm{Middlebury~Count}\)")
ax.set_xlabel("\(acc\)")

ax.set_xlim([0.5, 1])
ax.grid(True)

ax.text

ax.legend(loc=(0.06, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_acc_middlebury.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_acc_middlebury.png"))

In [ ]:
print(df_mtp[df_mtp[("CENSUS", "acc")]<0.7])

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_mtp[("CENSUS", "acc")], bins=50, range=(0.5,1))
mccnn_bin, mccnn_edges = np.histogram(df_mtp[("MCCNN", "acc")], bins=50, range=(0.5,1))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.axvline(0.9, color="k", linestyle=":")
ax.set_ylabel("\(\mathrm{MTP~Count}\)")
ax.set_xlabel("\(acc\)")

ax.set_xlim([0.5, 1])
ax.grid(True)

ax.legend(loc=(0.06, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_acc_mtp.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_acc_mtp.png"))

## Relative size

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_middlebury[("CENSUS", "s_rel")], bins=50, range=(0,0.5))
mccnn_bin, mccnn_edges = np.histogram(df_middlebury[("MCCNN", "s_rel")], bins=50, range=(0,0.5))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.set_ylabel("\(\mathrm{Middlebury~Count}\)")
ax.set_xlabel("\(s_{rel}\)")

ax.set_xlim([0., 0.5])
ax.grid(True)

ax.legend(loc=(0.4, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_s_rel_middlebury.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_s_rel_middlebury.png"))

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_mtp[("CENSUS", "s_rel")], bins=50, range=(0.,0.5))
mccnn_bin, mccnn_edges = np.histogram(df_mtp[("MCCNN", "s_rel")], bins=50, range=(0.,0.5))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.set_ylabel("\(\mathrm{MTP~Count}\)")
ax.set_xlabel("\(s_{rel}\)")

ax.set_xlim([0., 0.5])
ax.grid(True)

ax.legend(loc=(0.4, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_s_rel_mtp.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_s_rel_mtp.png"))

## Residual error

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_middlebury[("CENSUS", "eps")], bins=50, range=(0.,0.6))
mccnn_bin, mccnn_edges = np.histogram(df_middlebury[("MCCNN", "eps")], bins=50, range=(0.,0.6))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.set_ylabel("\(\mathrm{Middlebury~Count}\)")
ax.set_xlabel("\(\epsilon\)")

ax.set_xlim([0, 0.6])
ax.grid(True)

ax.legend(loc=(0.4, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_eps_middlebury.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_eps_middlebury.png"))

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_mtp[("CENSUS", "eps")], bins=50, range=(0,0.6))
mccnn_bin, mccnn_edges = np.histogram(df_mtp[("MCCNN", "eps")], bins=50, range=(0,0.6))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.set_ylabel("\(\mathrm{MTP~Count}\)")
ax.set_xlabel("\(\epsilon\)")

ax.set_xlim([0., 0.6])
ax.grid(True)

ax.legend(loc=(0.4, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_eps_mtp.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_eps_mtp.png"))

## Relative overestimation

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_middlebury[("CENSUS", "o_rel")], bins=50, range=(0.,1))
mccnn_bin, mccnn_edges = np.histogram(df_middlebury[("MCCNN", "o_rel")], bins=50, range=(0.,1))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.set_ylabel("\(\mathrm{Middlebury~Count}\)")
ax.set_xlabel("\(o_{rel}\)")

ax.set_xlim([0., 1])
ax.grid(True)

ax.legend(loc=(0.03, 0.8), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_o_rel_middlebury.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_o_rel_middlebury.png"))

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

census_bin, census_edges = np.histogram(df_mtp[("CENSUS", "o_rel")], bins=50, range=(0.,1))
mccnn_bin, mccnn_edges = np.histogram(df_mtp[("MCCNN", "o_rel")], bins=50, range=(0.,1))

ax.stairs(census_bin, census_edges, color=blue, linewidth=2, label="\(\mathrm{CENSUS}\)")
ax.stairs(mccnn_bin, mccnn_edges, color=orange, linewidth=2, label="\(\mathrm{MC-CNN}\)")

ax.set_ylabel("\(\mathrm{MTP~Count}\)")
ax.set_xlabel("\(o_{rel}\)")

ax.set_xlim([0., 1])
ax.grid(True)

ax.legend(loc=(0.06, 0.7), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "histogram_o_rel_mtp.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "histogram_o_rel_mtp.png"))

# MTP 278

In [ ]:
scene = "MTP_278"

root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/"
output_path = os.path.join(root_gt, cost_function, scene)
d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data = load_data_mtp(output_path)
img_left = rasterio.open(os.path.join(root_gt, cost_function, scene, "left_epipolar_image_gray.tif")).read(1)

In [ ]:
correct_intervals = (int_90_inf <= true_disp) & (true_disp <= int_90_sup)
img_mask = np.zeros(correct_intervals.shape + (4,))
img_mask[(~correct_intervals) & valid_data] = (1, 0, 0, 1)
vmin, vmax = min(np.nanmin(disp_map), np.nanmin(true_disp)), max(np.nanmax(disp_map), np.nanmax(true_disp))

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(img_left, cmap="gray", vmin=0, vmax=np.nanquantile(img_left, 0.99))
axes.axis("off")
plt.savefig(os.path.join(out_path, f"img_error_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"img_error_{scene}.png"))


fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(img_left, cmap="gray", vmin=0, vmax=np.nanquantile(img_left, 0.99))
axes.imshow(img_mask)
axes.axis("off")
plt.savefig(os.path.join(out_path, f"img_error_{scene}_err.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"img_error_{scene}_err.png"))


fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(true_disp, vmin=vmin, vmax=vmax)
axes.axis("off")
plt.savefig(os.path.join(out_path, f"img_error_{scene}_gt.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"img_error_{scene}_gt.png"))


fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(disp_map, vmin=vmin, vmax=vmax)
axes.axis("off")
plt.savefig(os.path.join(out_path, f"img_error_{scene}_disp.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"img_error_{scene}_disp.png"))

In [ ]:
df_mtp.index[60:80]

# MTP 153, MTP_6

In [ ]:
for scene in ["MTP_120", "MTP_153"]:

    root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/"
    output_path = os.path.join(root_gt, cost_function, scene)
    true_disp = rasterio.open(os.path.join(output_path, "left_epipolar_disp.tif")).read(1)
    true_disp[true_disp==0] = np.nan
    img_left = rasterio.open(os.path.join(root_gt, cost_function, scene, "left_epipolar_image_gray.tif")).read(1)

    fig, axes = plt.subplots(1, 1, figsize=(5, 5))
    axes.imshow(img_left, cmap="gray", vmin=0, vmax=np.nanquantile(img_left, 0.99))
    axes.axis("off")
    plt.savefig(os.path.join(out_path, f"img_{scene}.png"), dpi=250, bbox_inches='tight')
    crop_img(os.path.join(out_path, f"img_{scene}.png"))

    fig, axes = plt.subplots(1, 1, figsize=(5, 5))
    axes.imshow(true_disp, vmin=vmin, vmax=vmax)
    axes.axis("off")
    plt.savefig(os.path.join(out_path, f"img_{scene}_gt.png"), dpi=250, bbox_inches='tight')
    crop_img(os.path.join(out_path, f"img_{scene}_gt.png"))

# Good Middlebury

In [ ]:
year_dict["Piano-perfect"]

In [ ]:
root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/"
list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/CENSUS") if os.path.isdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/CENSUS/{k}")]
load_data = load_data_middlebury


#list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/CENSUS") if "MTP_"==k[:4]]
#root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/"
#load_data = load_data_mtp

In [ ]:
cost_function = "CENSUS"
scene = "Piano-perfect"

row = 900
col_inf, col_sup = 575, 1000

output_path = os.path.join(root_gt, cost_function, scene)
d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data = load_data(output_path)
img_left = rasterio.open(os.path.join("/work/CAMPUS/users/malinoro/Data/MiddleBury", year_dict["Piano-perfect"], scene, "im0_gray.png")).read(1)
true_disp[true_disp==0] = np.nan
correct_int = (true_disp<=int_90_sup) & (true_disp>=int_90_inf)
img_false_int = np.zeros(img_left.shape + (4,))
img_false_int[(~correct_int) & valid_data] = (1, 0, 0, 1)

img_false_int[row-15:row+15, col_inf:col_sup+1]=(0, 1, 1, 1)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,6))
ax.imshow(img_left, cmap="gray")
ax.axis("off")
plt.savefig(os.path.join(out_path, f"{scene}_{year_dict[scene]}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_{year_dict[scene]}.png"))

fig, ax = plt.subplots(1,1,figsize=(7,6))
ax.imshow(img_left, cmap="gray")
ax.imshow(img_false_int)
ax.axis("off")
plt.savefig(os.path.join(out_path, f"{scene}_{year_dict[scene]}_error.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_{year_dict[scene]}_error.png"))

In [ ]:

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), int_90_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), int_90_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)


ax.legend(loc=(0.75, 0.15), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, f"{scene}_{year_dict[scene]}_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_{year_dict[scene]}_row_{row}.png"))

# Bad Middlebury

In [ ]:
root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/"
list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/CENSUS") if os.path.isdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/CENSUS/{k}")]
load_data = load_data_middlebury


#list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/CENSUS") if "MTP_"==k[:4]]
#root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/"
#load_data = load_data_mtp

In [ ]:
cost_function = "CENSUS"
scene = "Sword2-perfect"

#row = 900
#col_inf, col_sup = 575, 1000

row = 1500
col_inf, col_sup = 1500, 2000

output_path = os.path.join(root_gt, cost_function, scene)
d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data = load_data(output_path)
img_left = rasterio.open(os.path.join("/work/CAMPUS/users/malinoro/Data/MiddleBury", year_dict["Piano-perfect"], scene, "im0_gray.png")).read(1)
true_disp[true_disp==0] = np.nan
correct_int = (true_disp<=int_90_sup) & (true_disp>=int_90_inf)
img_false_int = np.zeros(img_left.shape + (4,))
img_false_int[(~correct_int) & valid_data] = (1, 0, 0, 1)

img_false_int[row-15:row+15, col_inf:col_sup+1]=(0, 1, 1, 1)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,6))
ax.imshow(img_left, cmap="gray")
ax.axis("off")
plt.savefig(os.path.join(out_path, f"{scene}_{year_dict[scene]}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_{year_dict[scene]}.png"))

fig, ax = plt.subplots(1,1,figsize=(7,6))
ax.imshow(img_left, cmap="gray")
ax.imshow(img_false_int)
ax.axis("off")
plt.savefig(os.path.join(out_path, f"{scene}_{year_dict[scene]}_error.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_{year_dict[scene]}_error.png"))

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), int_90_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), int_90_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)


ax.legend(loc=(0.65, 0.01), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, f"{scene}_{year_dict[scene]}_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_{year_dict[scene]}_row_{row}.png"))

# Good MTP

In [ ]:
list_scenes = [k for k in os.listdir(f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/CENSUS") if "MTP_"==k[:4]]
root_gt = f"/work/CAMPUS/users/malinoro/outputs/CVPR_tests/MTP/"
load_data = load_data_mtp

In [ ]:
cost_function = "CENSUS"
scene = "MTP_291"

row = 750
col_inf, col_sup = 530, 950

output_path = os.path.join(root_gt, cost_function, scene)
d_min, d_max, true_disp, disp_map, int_90_inf, int_90_sup, validity_bitmap, valid_data = load_data(output_path)
img_left = rasterio.open(os.path.join(root_gt, cost_function, scene, "left_epipolar_image_gray.tif")).read(1)

true_disp[true_disp==0] = np.nan
correct_int = (true_disp<=int_90_sup) & (true_disp>=int_90_inf)

In [ ]:
d_min, d_max

In [ ]:
custom_mask = np.ones(valid_data.shape, dtype=np.bool_)
custom_mask[250:375, 450:550] = False
custom_mask[1250:1375, 800:1175] = False
custom_mask[1250:1750, 1250:1500] = False
img_false_int = np.zeros(img_left.shape + (4,))
img_false_int[(~correct_int) & valid_data & custom_mask] = (1, 0, 0, 1)
img_false_int[row-5:row+5, col_inf:col_sup+1]=(0, 1, 1, 1)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,6))
ax.imshow(img_left, cmap="gray", vmin=np.nanquantile(img_left, 0.01), vmax=np.nanquantile(img_left, 0.99))
ax.axis("off")
plt.savefig(os.path.join(out_path, f"{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}.png"))

fig, ax = plt.subplots(1,1,figsize=(7,6))
ax.imshow(img_left, cmap="gray", vmin=np.nanquantile(img_left, 0.01), vmax=np.nanquantile(img_left, 0.99))
ax.imshow(img_false_int)
ax.axis("off")
plt.savefig(os.path.join(out_path, f"{scene}_error.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_error.png"))

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), int_90_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), int_90_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)


ax.legend(loc=(0.5, 0.15), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, f"{scene}_error_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_error_row_{row}.png"))

# TMP

In [ ]:
for k in os.listdir("/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD"):
    print(k)
    lidar_hd = rasterio.open(f"/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD/{k}/{k}.tif")
    print(lidar_hd.meta)
    print(np.unique(rasterio.open("/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD/Bordeaux/Bordeaux.tif").read(1)))
    print()

In [ ]:
lidar_hd = rasterio.open("/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD/Bordeaux/Bordeaux.tif")
lidar_hd.meta

In [ ]:
np.unique(rasterio.open("/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD/Bordeaux/Bordeaux.tif").read(1))